# Simulacion de Trayectoria Ascendente y Descendete de un Globo-sonda


<h1> 1. Preparación del Notebook </h1>  <a name="prework"></a>

Se cargan las biblioteca o módulos necesarios para la simulación.

In [12]:
# =====================================================================================================
#   Módulos utilizadas en simulación:
# =====================================================================================================
# Generación de gráficos a partir de listas o arrays.
import matplotlib.pyplot as plt
import numpy as np                  # Manipulación de vectores, matrix y tensores
import pandas as pd

import sympy as sp
import scipy as scp
from scipy.integrate import solve_ivp

import getgfs
import pyproj


# 2 Descripición de la Misión  <a name="mision"></a>

<figure style="text-align:center;">
    <img src = "/home/osmin-ubuntu/proyecto_stratoballoon/document/figures/01_etapas de la mision.png"
        width="600" height="400"">
    <figcaption> Fig. 1 Fase de Vuelo.</figcaption>
</figure>

Las operaciones con globos de gran altitud se resumen en 5 fases
- **Fase 1** es el lanzamiento de la sonda desde unas coordenadas predefinidas.
- **Fase 2** es el ascenso de la sonda hasta una altitud de varios kilómetros determinado por el fabricante del globo [7] y el peso de los instrumentos.
- **Fase 3** es la explosión del globo
- **Fase 4** es el descenso con solo el paracaídas y la carga útil.
- **Fase 5** que es la recuperación de la sonda con ayuda de los instrumentos de navegación

## 2.1 Propiedades, estado y perfil  <a name="propiedades"></a>

Se define los pararámetros del sistema que incluye constantes, valores de entradas, propiedades y perfil de la misión.

Estos valores se retoman a lo largo de la programación para el cálculo del lanzamiento, operación y caída del vuelo para un High-altitude Balloon. 

> *Nota:* algunas propiedades se definen espeficicamente en cada bloque de programación.  

In [13]:
# =====================================================================================================
#                                    LANZAMIENTO DE LA SONDA:
#                                Autodromo Internacional El Jabalí
#                               https//goo.gl/maps/sARALqQY2UCwtPMZ7
# =====================================================================================================
# Coordenadas geográficas (latitud, longitud y altitud)
lat_launch = 13.808825
lon_launch = -89.328988
alt_launch = 504
# Fecha, Hora, y objetivo de la misión
date_launch = '2023-06-10 '
time_launch = '10:00:00'
target_burst_altitude = 30000

# =====================================================================================================
#               Parámetros iniciales de atmosféricos
# =====================================================================================================

M_He = 4.0026/1000  # kg/mol Masa molar del Helio
M_air = 28.97/1000  # kg/mol Masa molar del Aire Seco
R_gas = 8.3144661815324  # m³ Pa / (K mol)
P_launch = 101325  # Pa
T_launch = 288.15  # K
g_launch = 9.80665  # m/s²

# =====================================================================================================
#                     Parámetros del Globo en base a Datasheet Kaymount HAB 1500
# =====================================================================================================
m_b = 1.5                               # Peso en kg
# ballon_launch_diameter = 1.85           # m
balloon_bursting_altitude = 34.2e3      # m
balloon_bursting_pressure = 630         # Pa
balloon_bursting_diameter = 9.44        # m

# Coeficiente de arrastre del globo (drag coefficient of balloon)
CD_b = 0.47
# C_dadded = 0.25         # c_Added coefficient

# =====================================================================================================
#                     Parámetros del Paracaidas en base a Datasheet Kaymount HAB-160V
# =====================================================================================================
# # parachute_weight = 0.18  # kg
parachute_diameter = 1.56  # m
Cd_pchute = 0.8     	# Parachute drag coefficient
# # Cd_spchute = 0.1    	# parachute side drag coefficient

# # voy a leer de donde sacó esto napoleon
# PSideL = 0.1	    	# parachute side length, flaps (m)
# parachute_work = 16e3   # m

# =====================================================================================================
#                     Parámetros del Payload
# =====================================================================================================
m_pl =  1.769


## Precalulos

### Transformación de Sistemas de Coordenadas

<figure style="text-align:center;">
    <img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/62/Ecef_coordinates.svg/559px-Ecef_coordinates.svg.png"
        width="500" height="400">
    <figcaption> Fig. 2 Sistemas de coordenadas</figcaption>
</figure>

Se utiliza las coordenas geodésicas WGS84 debido a que es la base de GPS y coordenadas cartesianas debido que es más facil plantear ecuaciones diferenciales en esta sistema. En Fig. 2 se muestra una ilustra como se establacen los sistemas de cooordenadas.

> 📝 **Nota:** en la siguiente celda de código, al realizar la transformación de WGS84 a ECEF, se especifica primero la longitud y luego la latitud para que la coordenada X se corresponda con la longitud y la coordenada Y se corresponda con la latitud.

En WGS84 la distancia vertical desde el elipsoide de referencia hasta el punto en cuestión en el sistema de coordenadas WGS84 se conoce como altitud geométrica o altitud elipsoida

In [14]:
# Objeto Conversosr de coordenadas 
lonlat_to_xy = pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:3857') # (Latitud y Longitud, XY cartesiano)
xy_to_lonlat = pyproj.Transformer.from_crs('EPSG:3857', 'EPSG:4326')

### Estimación de masa del Gas de elevación

A continuación, se desea calcular el diametro inicial del sistema y la masa que contendrá el globo en el punto de lanzamiento.  Asi que, por conservación de masa podemos afirmar que la materia en toda la trayectoria el sistema se no cambiará en función del tiempo hasta la explosición del globo.

$$
\textbf{Ley de Gases ideales:}
\newline
$$

$$
\begin{equation}
    PV = nRT  
    \hspace{8ex}  
    \tag 1
\end{equation}
\newline
$$

Por lo tanto, en base a la Ecua. 1 y conservación de masa, se tiene que: 

$$
\begin{equation}
    \frac{P_{1} V_{1}}{n_{1} T_{1}} 
    = 
    \frac{P_{2} V_{2}}{n_{2} T_{2}}
    = R
    \nonumber
\end{equation}
$$

<br>

$$
\begin{equation}
    \left. \begin{matrix}
    \frac{P_{1} V_{1}}{n_{1} T_{1}} = \frac{P_{2} V_{2}}{n_{2} T_{2}}
    \\ 
    \\
    n = \frac{m}{M} = cte
    \end{matrix}\right\}

    \rightarrow 

    \frac{P_{1} V_{1}} {T_{1}} = \frac{P_{2} V_{2}} {T_{2}} 
    \nonumber
\end{equation}
$$

<br>

$$
\begin{equation}
    \frac{P_{1} V_{1}} {T_{1}} = \frac{P_{2} V_{2}} {T_{2}} 
    \hspace{4ex}  
    \tag 2
\end{equation}
$$

> 📝  **NOTA:** Entiendase los subindices asi, subindice 1 significa estado del gas en el punto de lanzamiento, punto 1; y subindice 2 como el punto final al cual la sonda debe de alcanzar, punto 2.

**En el punto 1** conocemos la presión y temperatrura, se desea conocer el volumen inicial o volumen de llenado del globo, para luego calcular la masa. **En el punto 2** conocemos la presión, volumen, temperatura final que se toman en base a lo que dice la hoja técnica que puede aguantar el globo seleccionado. Por lo tanto, en base a la Ecua 2 se realiza.


La funcion de estmación si no se especifica lo contrario deberia tener de entrada un parametro por default, que seria la altitud máxima del fabricante que lelga el globo.

Además se debe de poner que el fabricante la altitud, presion y datos asi lo saca de la ISA jejej 


In [15]:
def estimacion_gas_elevacion():
    # Volumen 1 incial [m^3]
    v1 = sp.Symbol("v1")  # Volumen de llenado

    # Volumen 2 final [m^3]
    # Volumen de una Esfera, Volumen FINAL
    v2 = (4/3)*np.pi*np.power(balloon_bursting_diameter/2, 3)

    # ∆ presiones 1,2 (inicial, final) [Pa]
    p1, p2 = P_launch, balloon_bursting_pressure

    # ∆ temperaturas 1,2 (inicial, final) [K]
    # t2_cal = atm_
    t1, t2 = T_launch, 234.810

    # CALCULO DE VOLUMEN
    eqn = sp.Eq((p1*v1)/t1, (p2*v2)/t2)  # Ecua. 2
    v1 = sp.solve(eqn)
    volumen_inicial = v1[0]

    # CALCULO DE DIAMETRO
    d1 = sp.Symbol("d1")  # Diametro de llenado
    # Volumen de una Esfera, Volumen INICIAL
    eqn = sp.Eq(volumen_inicial, (4/3)*np.pi*np.power(d1/2, 3))
    diametro_inicial = sp.solve(eqn)
    diametro_inicial = diametro_inicial[0]

    # CALCULO DE MASA DEL GAS DE ELEVACIÓN
    m_gas = sp.Symbol('m_gas')
    eqn = sp.Eq(p1*v1[0], (m_gas/M_He)*R_gas*t1)
    masa_gas = sp.solve(eqn)
    masa_gas = masa_gas[0]

    # lista con las estimaciones
    return [diametro_inicial, volumen_inicial, masa_gas]


estimate_gas = estimacion_gas_elevacion()
print(
    f'Diametro inicial: {estimate_gas[0]} [m] \nVolumen inicial: {estimate_gas[1]} [m^3] \nMasa del Helio es: {estimate_gas[2]} [Kg]')

ballon_launch_diamete, ballon_launch_volumen, m_He = estimate_gas[
    0], estimate_gas[1], estimate_gas[2]


Diametro inicial: 1.85843475385261 [m] 
Volumen inicial: 3.36078381949232 [m^3] 
Masa del Helio es: 0.568913691920873 [Kg]


## 3.1 Gravedad

La intensidad del campo gravitario o sencillamente gravedad a medida que se asciende no es constante, disminuye en función de la altitud. Esta disminución se puede expresar de la siguiente manera:

$$
\begin{equation}
g = g_{0} \left(\frac{r_e}{r_e + z}\right)^2
\nonumber
\end{equation}
$$

Donde 
- $g_{0}$ es la aceleración $ 9.80665 m/s{^2} $ debida a la gravedad en la superficie de la Tierra
- $r_e$ es el radio promedio de la Tierra igual a $ 6,371,000 m $. Asuminedo que es una esfera nuestro globoterraqueo. 
- $z$ es la altitud geométrica, es decir, altitud sobre la superficie de la Tierra o con respecto al nivel del mar.

Se usa una aproximación para la gravedad estándar y puede existir variaciones en la gravedad real debido a muchos factores, acá se hace una simplificación y homogenización para fines prácticos. A continuación el código:

In [16]:
def earth_gravity_Ln(altitude):
    z = altitude
    G0 = 9.80665  # Gravedad estándar ASL [m/s^2]
    RE = 6371000  # Radio medio de la tierra.
    g = G0 * ((RE / (RE + z)) ** 2)
    return g

# print(earth_gravity(h))


## 3.2 Temperatura, presión y densidad  <a name="atmosferico"></a>


Se utiliza el modelo atmosférico estándar (The International Standard Atmosphere, por sus siglas ISA), con este modelo se calcula la temperatura, la presión y densidad de la atmósfera [4]. **Todo lo generado con este modelo atmosférico es función de la altitud.**

Como el objetivo de la misión es llegar a los 30 Km (que es lo que suelen llegar los HAB), el modelo se divide en tres regiones diferentes como se ve en Talba 1 y Talba 2.

<center>
<b> Tabla 1. Constantes del ISA </b>

| **Capa** | $z_0$ [m] | $T_0$ [K] | $\lambda_0$ [K/m] | $P_0$ [Pa] |
|:----------:|:---------:|:---------:|:-----------------:|:----------:|
| 1          | 0         | 288.15    | -0.0065           | 101,325.00 |
| 2          | 11,019    | 216.65    | -                 | 22,632.10  |
| 3          | 20,063    | 216.65    | 0.0010            | 5,474.89   |
| 4          | 32,162    | 228.65    | 0.0028            | 868.02     |
| 5          | 47,359    | 270.65    | -                 | 110.91     |

</center>

<br>

<center>
<b> Tabla 2. Fórmulas de ISA </b>

| **Capa** |         **Temperatura [K]**         |                                **Presión [Pa]**                                | **Densidad [kg/m³]** |
|:--------:|:-----------------------------------:|:------------------------------------------------------------------------------:|:---------------------:|
|     1    | $ T_{0} + \lambda_{0} (z - z_{0}) $ | $ P_0 \; \left( \frac{T_0}{T} \right)^{ g(z) \; M_{air} / (R \; \lambda_0)} $  | $ \frac{P}{T R_{s}} $ |
|     2    |              $ T_{0} $              |           $ P_0 \; e^{- g(z) \; M_{air} \cdot (z - z_0) / (R \; T)} $          | $ \frac{P}{T R_{s}} $ |
|     3    | $ T_{0} + \lambda_{0} (z - z_{0}) $ |  $ P_0 \; \left( \frac{T_0}{T} \right)^{ g(z) \; M_{air} / (R \; \lambda_0)} $ | $ \frac{P}{T R_{s}} $ |
|     4    | $ T_{0} + \lambda_{0} (z - z_{0}) $ |  $ P_0 \; \left( \frac{T_0}{T} \right)^{ g(z) \; M_{air} / (R \; \lambda_0)} $ | $ \frac{P}{T R_{s}} $ |
|     5    |              $ T_{0} $              |           $ P_0 \; e^{- g(z) \; M_{air} \cdot (z - z_0) / (R \; T)} $          | $ \frac{P}{T R_{s}} $ |

</center>

El valor de las constantes de la tabla 1 se configuran a continuación:

Algunos parámetros característicos de las zonas de la atmosfera. El parámetro $ z_{0} $ es la altitud de la zona, $ R = 8.31432 $ J/(K mol)  y $ R_{s} = 287.04 $ J / (K kg) son constantes de los gases ideales, $ M_{air} = 0.0289644 $ kg/mol, $ g(z) = g_{0}(R_{E}/(R_{E} + z))^{2} $ es la aceleración de la gravedad, $g_{0} = 9,80665 $ $m/s^{2}$ es la constante estándar de aceleración de la gravedad y $ R_{E} = 6,371×10^{6} $ es la constante estándar de aceleración de la gravedad y $ R_{E} = 6,371×10^{6} $ m es el radio medio de la Tierra.

### Temperatura del Aire



In [17]:
def atm_tmp_Ln(altitude):
    z = altitude

    if z < 11019:
        z0 = 0
        T0 = 288.15
        lambda0 = -0.0065
        P0 = 101325.0

        l1 = T0 + lambda0 * (z - z0)
        return l1

    elif z < 20063:
        z0 = 11019
        T0 = 216.65

        l2 = T0
        return l2

    elif z < 32162:
        z0 = 20063
        T0 = 216.65
        lambda0 = 0.0010
        P0 = 5474.89

        l3 = T0 + lambda0 * (z - z0)
        return l3

    elif z < 47359:
        z0 = 32162
        T0 = 228.65
        lambda0 = 0.0028
        P0 = 868.02

        l4 = T0 + lambda0 * (z - z0)
        return l4



### Presion del Aire


In [18]:
def atm_pressure_Ln(altitude):
    z = altitude
    t = atm_tmp_Ln(z)
    g = earth_gravity_Ln(z)

    if z < 11019:
        z0 = 0
        T0 = 288.15
        lambda0 = -0.0065
        P0 = 101325.0

        l1 = P0 * ((T0 / t) ** (g * 0.0289644 / (8.31432 * lambda0)))
        return l1

    elif z < 20063:
        z0 = 11019
        T0 = 216.65
        P0 = 22632.10

        l2 = P0 * np.exp(- g * 0.0289644 *
                         (z - z0) / (8.31432 * t))
        return l2

    elif z < 32162:
        z0 = 20063
        T0 = 216.65
        lambda0 = 0.0010
        P0 = 5474.89

        l3 = P0 * ((T0 / t) **
                   (g * 0.0289644 / (8.31432 * lambda0)))
        return l3

    elif z < 47359:
        z0 = 32162
        T0 = 228.65
        lambda0 = 0.0028
        P0 = 868.02

        l3 = P0 * ((T0 / t) **
                   (g * 0.0289644 / (8.31432 * lambda0)))
        return l3




### Densidad del Aire

In [19]:
def atm_rho_Ln(altitude):
    z = altitude
    p = atm_pressure_Ln(z)
    t = atm_tmp_Ln(z)

    if z < 47359:
        return p / (t * 287.04)



# 4 Modelo Geométria

A medida el globo sonda asciende, el aire se volverá menos denso y el gas en su interior se expandirá. Lo que provocará que el diametro/radio del globo se haga más grande, para calcular podemos aplicar la Ley de los Gases ideales, la ISA y geometría. Entonces para dejar eso en función de la altitud tenemos:

$
\begin{equation}
    P(z) \; V = n \; R \; T(z) 
\end{equation}
 $ 

Donde $ P(z) $ es la presión interna del gas, que será igual a la presión externa atmosférica; $ n $ es la cantidad de moles de Helio y $ T(z) $ la temperatura atmosférica igual que la temperatura del gas, por ultimo $ R $ es la constante de los gases ideales. Con lo anterior y aplicando geometría se tiene:  

$$
\begin{equation}
    \left. \begin{matrix}
    \begin{equation} P \; V = n \; R \; T  \hspace{4ex}  \tag x \end{equation}
    \\ 
    \\
    \begin{equation} n = m/M  \hspace{6ex}  \tag y \end{equation}
    \\ 
    \\
    \begin{equation} V = \frac{4}{3} \pi r^{3} \hspace{4ex}  \tag z \end{equation} 

    \end{matrix}\right\}

    \rightarrow 

    r = \sqrt[3]{\frac{ 3 \; m \; R \; T }{ 4 \pi \; P \; M }}  
    \nonumber
\end{equation}
$$

Se ha asumido que el globo tien forma esférica, por lo tanto los cálculos necesario para la fueraza de empuje y arrastre los cuales involucra volumen y area respectivametne se puede resolver de la siguiente manera:

### Volumen 

El globo sonda se definirá como una esfera para calcular su volumen, a continuación se expresa:

$ V_{globo} = \frac{4}{3} \pi r^{3}_{0} $

### Area Transversal

El área transversal de una esfera es la superficie de corte de una esfera por un plano perpendicular a su eje. En el caso de una esfera perfecta, la sección transversal será un círculo con un diámetro igual al diámetro de la esfera. Por lo tanto, se tiene que:

$A_{trans} = \pi r^{2} = \frac{\pi d^2}{4}$


In [20]:
def radio_crecimiento_globo_Ln(altitude):
    z = altitude
    t = atm_tmp_Ln(z)
    p = atm_pressure_Ln(z)
    # print(z, t, p)

    r = np.power((3 * m_He * R_gas * t) / (4 * np.pi * p * M_He), 1/3)
    return r


# 5 Modelo Dinámico


## 4.2.1 **Movimiento vertical ascenso:**

El movimiento se expresa mediante esta ecuación:
$$
\begin{equation}
m \frac{\mathrm{d^2} z }{\mathrm{d} t^2}  =  F_{Empuje} - W_{Sonda} - F_{Arrastre}
\nonumber
\end{equation}
$$

Ampliando la ecuación se tiene: 
$$
 ( m_{b} + m_{gas} + m_{pl} ) \; \frac{\mathrm{d^2} z }{\mathrm{d} t^2}  =  \rho_{atm} (z) \; V_{b} \; g (z) -( m_{b} + m_{pl} ) \; g(z) - \frac{1}{2} C_{D} \; \rho_{atm} \; S_{b} \; \left ( \frac{\mathrm{d} z }{\mathrm{d} t} \right )^{2}
$$



### 4.2.3 Viento

$$
\begin{equation}
    \frac{\mathrm{d} x }{\mathrm{d} t} = - w(z) \cdot  \cos(\phi(z))
    \hspace{8ex}  
    \tag 1
\end{equation}
\newline
$$

<br>

$$
\begin{equation}
    \frac{\mathrm{d} y }{\mathrm{d} t} = - w(z) \cdot \sin(\phi(z))
    \hspace{8ex}  
    \tag 1
\end{equation}
\newline
$$



In [21]:
f = getgfs.Forecast("0p25")
u_interp, v_interp = f.get_windprofile(
    date_launch+time_launch, lat_launch, lon_launch)



# Funcionalidad del Modelo


 ## Ascenso del sistema
 
Considerando la ecuación, vistas con anterioridad:
$$
\begin{align}
\frac{\mathrm{d^2} z }{\mathrm{d} t^2}  &=  \frac{ \rho_{atm} (z) \; V_{b} (z) \; g (z) -( m_{b} + m_{pl} ) \; g (z) - \frac{1}{2} C_{D} \; \rho_{atm} \; S_{b} (z) \; \left ( \frac{\mathrm{d} z }{\mathrm{d} t} \right )^{2} }{\left( m_{b} + m_{gas} + m_{pl}  \right)}  && \text{ascenso} \tag a \\

\frac{\mathrm{d} x }{\mathrm{d} t} &= - w(z) \cdot  \cos(\phi(z))    && \text{viento x} \tag b \\

\frac{\mathrm{d} y }{\mathrm{d} t} &= - w(z) \cdot \sin(\phi(z))     && \text{viento y} \tag c \\

\end{align}
$$


Lo que podemos ver es que:
$$
\frac{\mathrm{d^2} z }{\mathrm{d} t^2}  = f(t, z)
$$

In [22]:
# Función que define la ecuación diferencial (ED)
def motion_equation_ascenso(t, S):
    z, vz, x, y = S

    # Funciones de utilidad para las ED's
    g = earth_gravity_Ln(z)
    rho = atm_rho_Ln(z)    
    r = radio_crecimiento_globo_Ln(z)
    V_b = (4 / 3) * np.pi * (r**3)
    S_b = np.pi * (r ** 2)

    # Se definen las ED's:
    dx_dt = - v_interp(z)
    dy_dt = - u_interp(z)

    # Recalculamo las componentes en la nueva posicion
    

    return [vz,
            (rho * V_b * g - (m_b + m_pl) * g - 0.5 * CD_b * rho * S_b * vz**2) / (m_b + m_He + m_pl),
            dx_dt, dy_dt]


Configuramos las condiciones iniciales

In [23]:
# Condiciones iniciales de la posición:
z_0 = alt_launch
vz_0 = 0.0
x_0, y_0 = lonlat_to_xy.transform(lon_launch, lat_launch) # Convercion de coordenadas

S_0 = [z_0, vz_0, x_0, y_0]

# Intervalo de tiempo de integración:
t_start = 0
t_end = 10000
t_step = 1

t_eval = np.arange(t_start, t_end + t_step, t_step)


Configuramos la función que determina cuanso se debe terminar la 

In [24]:
# Función que termina la integración  cuando se cumple el Objetivo de la Altitud
def limit_ascenso(t, S):
    return S[0] - target_burst_altitude

limit_ascenso.terminal = True
limit_ascenso.direction = 0

# Resolución de ecuación diferencial (ED)
solution_ascenso = solve_ivp(motion_equation_ascenso, t_span=(t_start, t_end), y0=S_0, method='LSODA',  events=limit_ascenso, t_eval=t_eval)

# Resultados de solución ED
print(solution_ascenso)


  message: 'A termination event occurred.'
     nfev: 146
     njev: 12
      nlu: 12
      sol: None
   status: 1
  success: True
        t: array([   0,    1,    2, ..., 6663, 6664, 6665])
 t_events: [array([6665.89606407])]
        y: array([[ 5.04000000e+02,  5.05013199e+02,  5.07464428e+02, ...,
         2.99810095e+04,  2.99875658e+04,  2.99941233e+04],
       [ 0.00000000e+00,  1.90048521e+00,  2.84825343e+00, ...,
         6.52951257e+00,  6.53059422e+00,  6.53167617e+00],
       [ 1.53719137e+06,  1.53719101e+06,  1.53719064e+06, ...,
         1.54399169e+06,  1.54398897e+06,  1.54398625e+06],
       [-3.27857352e+07, -3.27857352e+07, -3.27857353e+07, ...,
        -3.27422203e+07, -3.27421934e+07, -3.27421664e+07]])
 y_events: [array([[ 3.00000000e+04,  6.53264593e+00,  1.54398381e+06,
        -3.27421422e+07]])]


Se reconvierten de las coordenadas del plano XY a coordenadas Cartograficas

In [25]:
# Convertir de latitud y longitud a las coordenadas del plano XY cartesiano
solution_ascenso.y[2], solution_ascenso.y[3] = xy_to_lonlat.transform(solution_ascenso.y[2], solution_ascenso.y[3])

print(solution_ascenso.y[2][-1], solution_ascenso.y[3][-1])


-89.32438875980357 13.869864472677106


Hacemos el recalculo de la variables debibo a que scipy no devuelve los valores de la ISA, tomamos la altura que si devuelve scipy y lo ingresamos en los calculos de temperatura, presión y densidad

In [27]:
# # Vemos la generalidades de como se resolvió
time_ascenso = np.array(solution_ascenso.t)
z_values_ascenso = np.array(solution_ascenso.y[0])
dz_dt_values_ascenso = np.array(solution_ascenso.y[1])
longitud_ascenso = np.array(solution_ascenso.y[2])
latitud_ascenso = np.array(solution_ascenso.y[3])

# Definir un array para almacenar los resultados
gravedad_ascenso = np.empty(0)
temperatura_ISA_ascenso = np.empty(0)
presion_ISA_ascenso = np.empty(0)
densidad_ISA_ascenso = np.empty(0)
diametro_balloon_ascenso = np.empty(0)
viento_u_ascenso = np.empty(0)
viento_v_ascenso = np.empty(0)
estado_ascenso = np.empty(0)

for valor in z_values_ascenso:

    # Valores independientes a la capa donde se encuentre
    gravedad_ascenso = np.append(gravedad_ascenso, earth_gravity_Ln(valor))

    temperatura_ISA_ascenso = np.append(
        temperatura_ISA_ascenso, atm_tmp_Ln(valor))

    presion_ISA_ascenso = np.append(
        presion_ISA_ascenso, atm_pressure_Ln(valor))

    densidad_ISA_ascenso = np.append(densidad_ISA_ascenso, atm_rho_Ln(valor))

    diametro_balloon_ascenso = np.append(
        diametro_balloon_ascenso, radio_crecimiento_globo_Ln(valor)*2)
    
    viento_u_ascenso = np.append(
        viento_u_ascenso, u_interp(valor))
    
    viento_v_ascenso = np.append(
        viento_v_ascenso, v_interp(valor))
    
    estado_ascenso = np.append(estado_ascenso, True)


In [28]:
# Creando un DataFrame de Pandas con las resultados de los parámetros

df_a = pd.DataFrame({
    # 'Fecha': fecha,
    'tiempo': time_ascenso,
    'latitude': latitud_ascenso,
    'longitude': longitud_ascenso,
    'altitud': z_values_ascenso,
    's_vertical': dz_dt_values_ascenso,
    'viento_u': viento_u_ascenso,
    'viento_v': viento_v_ascenso,
    'temperatura': temperatura_ISA_ascenso,
    'presion': presion_ISA_ascenso,
    'densidad': densidad_ISA_ascenso,
    'gravedad': gravedad_ascenso,
    'diameter_balloon': diametro_balloon_ascenso,
    # 'estado': np.ones_like(estado_ascenso, dtype=bool)
})

# guardar el DataFrame del conjunto de ndarray como archivo .csv
df_a.to_csv(
    '/home/osmin-ubuntu/proyecto_stratoballoon/code/data/parameters_ascenso.csv', index=False)

# Resultados al guardar el DataFrame
df_a.round(3)

,tiempo,latitude,longitude,altitud,s_vertical,viento_u,viento_v,temperatura,presion,densidad,gravedad,diameter_balloon
0,0,13.809,-89.329,504.000,0.000,0.039,0.361,284.874,95415.967,1.167,9.805,1.88881997325474
1,1,13.809,-89.329,505.013,1.900,0.039,0.361,284.867,95404.378,1.167,9.805,1.88888189688063
2,2,13.809,-89.329,507.464,2.848,0.039,0.362,284.851,95376.344,1.166,9.805,1.88903172261734
3,3,13.809,-89.329,510.500,3.168,0.039,0.363,284.832,95341.638,1.166,9.805,1.88921728824262
4,4,13.809,-89.329,513.722,3.257,0.039,0.364,284.811,95304.814,1.166,9.805,1.88941427324323
...,...,...,...,...,...,...,...,...,...,...,...,...
6661,6661,13.870,-89.324,29967.900,6.527,-29.216,3.628,226.555,1205.871,0.019,9.715,7.51283992994875
6662,6662,13.870,-89.324,29974.454,6.528,-29.217,3.639,226.561,1204.695,0.019,9.715,7.51535698606150
6663,6663,13.870,-89.324,29981.009,6.530,-29.218,3.650,226.568,1203.520,0.019,9.715,7.51787520121832
6664,6664,13.870,-89.324,29987.566,6.531,-29.220,3.661,226.575,1202.345,0.018,9.715,7.52039457598436


## Descenso del Sistema

Considerando la ecuación, vistas con anterioridad:
$$
\begin{align}
\frac{\mathrm{d^2} z }{\mathrm{d} t^2}  &=   \frac{\frac{1}{2} C_{D} \; \rho_{atm} \; S_{p}} {( m_{b} + m_{pl} ) } \left ( \frac{\mathrm{d} z }{\mathrm{d} t} \right )^{2} - g(z)   && \text{descenso} \tag a \\

\frac{\mathrm{d} x }{\mathrm{d} t} &= - w(z) \cdot  \cos(\phi(z))    && \text{viento x} \tag b \\

\frac{\mathrm{d} y }{\mathrm{d} t} &= - w(z) \cdot \sin(\phi(z))     && \text{viento y} \tag c \\

\end{align}
$$


In [29]:
# Función que define la ecuación diferencial
def motion_equation_descenso(t, S):
    z, vz, x, y = S

    # Funciones de utilidad para las ED's
    g = earth_gravity_Ln(z)
    rho = atm_rho_Ln(z)
    S_p = np.pi * ((parachute_diameter/2) ** 2)

    # Se definen las ED's:
    dx_dt = - v_interp(z)
    dy_dt = - u_interp(z)

    return [vz, -g + ((0.5 * Cd_pchute * rho * S_p) / (m_b + m_pl)) * vz**2,
            dx_dt, dy_dt]


Configuramos las condiciones iniciales, se toman de entrada los ultimos datos de posción y tiempo de la simulación de ascenso.

In [32]:
# Condiciones iniciales de la posición:
z_0 = z_values_ascenso[-1]
vz_0 = dz_dt_values_ascenso[-1]
x0, y0 = lonlat_to_xy.transform(longitud_ascenso[-1], latitud_ascenso[-1]) # Convercion de coordenadas ultimas de simualación ascenso
S_0 = [z_0, vz_0, x0, y0]

# Intervalo de tiempo de integración:
t_start = time_ascenso[-1]
t_end = time_ascenso[-1] + 2500
t_step = 1
t_eval = np.arange(t_start, t_end + t_step, t_step)


In [33]:
# Función que termina la integración  uando se cumple que se llega al limite de la capa 1 de la Atomosfera
def limit_descenso(t, S):
    return S[0] - 0

limit_descenso.terminal = True
limit_descenso.direction = 0

# Resolusión de ecuación diferencial
solution_descenso = solve_ivp(motion_equation_descenso, t_span=(
    t_start, t_end), y0=S_0, method='LSODA', events=limit_descenso, t_eval=t_eval)

# Mostramos solución
print(solution_descenso)


  message: 'A termination event occurred.'
     nfev: 208
     njev: 14
      nlu: 14
      sol: None
   status: 1
  success: True
        t: array([6665, 6666, 6667, ..., 9017, 9018, 9019])
 t_events: [array([9019.42078573])]
        y: array([[ 2.99941233e+04,  2.99958293e+04,  2.99879406e+04, ...,
         1.41720679e+01,  8.31651096e+00,  2.46268734e+00],
       [ 6.53167617e+00, -3.13233204e+00, -1.25471055e+01, ...,
        -5.85432598e+00, -5.85272018e+00, -5.85111609e+00],
       [ 1.54398625e+06,  1.54398258e+06,  1.54397891e+06, ...,
         1.54654790e+06,  1.54654754e+06,  1.54654717e+06],
       [-3.27421664e+07, -3.27421372e+07, -3.27421079e+07, ...,
        -3.27312085e+07, -3.27312086e+07, -3.27312086e+07]])
 y_events: [array([[ 5.13779647e-12, -5.85044163e+00,  1.54654702e+06,
        -3.27312086e+07]])]


Se reconvierten de las coordenadas del plano XY a coordenadas Cartograficas

In [34]:
# Convertir de latitud y longitud a las coordenadas del plano XY cartesiano
solution_descenso.y[2], solution_descenso.y[3] = xy_to_lonlat.transform(solution_descenso.y[2], solution_descenso.y[3])

print(solution_descenso.y[2][-1], solution_descenso.y[3][-1])

-89.32322707069765 13.892869630869976


In [37]:
# # Vemos la generalidades de como se resolvió
time_descenso = np.array(solution_descenso.t)
z_values_descenso = np.array(solution_descenso.y[0])
dz_dt_values_descenso = np.array(solution_descenso.y[1])
longitud_descenso = np.array(solution_descenso.y[2])
latitud_descenso = np.array(solution_descenso.y[3])

# Definir un array para almacenar los resultados
gravedad_descenso = np.empty(0)
temperatura_ISA_descenso = np.empty(0)
presion_ISA_descenso = np.empty(0)
densidad_ISA_descenso = np.empty(0)
diametro_balloon_descenso = np.empty(0)
viento_u_descenso = np.empty(0)
viento_v_descenso = np.empty(0)
estado_descenso = np.empty(0)

for valor in z_values_descenso:

    # Valores independientes a la capa donde se encuentre
    gravedad_descenso = np.append(gravedad_descenso, earth_gravity_Ln(valor))

    temperatura_ISA_descenso = np.append(
        temperatura_ISA_descenso, atm_tmp_Ln(valor))

    presion_ISA_descenso = np.append(
        presion_ISA_descenso, atm_pressure_Ln(valor))

    densidad_ISA_descenso = np.append(densidad_ISA_descenso, atm_rho_Ln(valor))

    diametro_balloon_descenso = np.append(diametro_balloon_descenso, 0)

    viento_u_descenso = np.append(
        viento_u_descenso, u_interp(valor))
    
    viento_v_descenso = np.append(
        viento_v_descenso, v_interp(valor))

    estado_descenso = np.append(estado_descenso, False)


In [38]:
# Creando un DataFrame de Pandas con las resultados de los parámetros

df_d = pd.DataFrame({
    # 'Fecha': fecha,
    'tiempo': time_descenso,
    'latitude': latitud_descenso,
    'longitude': longitud_descenso,
    'altitud': z_values_descenso,
    's_vertical': dz_dt_values_descenso,
    'viento_u': viento_u_descenso,
    'viento_v': viento_v_descenso,
    'temperatura': temperatura_ISA_descenso,
    'presion': presion_ISA_descenso,
    'densidad': densidad_ISA_descenso,
    'gravedad': gravedad_descenso,
    'diameter_balloon': diametro_balloon_descenso,
    # 'estado': np.zeros_like(estado_descenso, dtype=bool),
})

# guardar el DataFrame del conjunto de ndarray como archivo .csv
df_d.to_csv(
    '/home/osmin-ubuntu/proyecto_stratoballoon/code/data/parameters_descenso.csv', index=False)

# Resultados al guardar el DataFrame
df_d.round(3)

,tiempo,latitude,longitude,altitud,s_vertical,viento_u,viento_v,temperatura,presion,densidad,gravedad,diameter_balloon
0,6665,13.870,-89.324,29994.123,6.532,-29.221,3.672,226.581,1201.172,0.018,9.715,0.0
1,6666,13.870,-89.324,29995.829,-3.132,-29.221,3.675,226.583,1200.867,0.018,9.715,0.0
2,6667,13.870,-89.324,29987.941,-12.547,-29.220,3.662,226.575,1202.278,0.018,9.715,0.0
3,6668,13.870,-89.324,29971.065,-21.005,-29.216,3.633,226.558,1205.303,0.019,9.715,0.0
4,6669,13.870,-89.324,29946.396,-28.060,-29.212,3.591,226.533,1209.739,0.019,9.715,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2350,9015,13.893,-89.323,25.888,-5.858,0.032,0.373,287.982,101014.388,1.222,9.807,0.0
2351,9016,13.893,-89.323,20.029,-5.856,0.032,0.373,288.020,101084.617,1.223,9.807,0.0
2352,9017,13.893,-89.323,14.172,-5.854,0.032,0.373,288.058,101154.866,1.223,9.807,0.0
2353,9018,13.893,-89.323,8.317,-5.853,0.032,0.373,288.096,101225.133,1.224,9.807,0.0


 # 5. Resultados    <a name="resultados"> </a>
   


## Datos de entrada

Estos son los datos de inicio y se exportan a una tabla en latex para llamarlo desde Overleaf.


Ponerle al la tabla un nombre como 

In [39]:
# Crear el dataframe
df = pd.DataFrame({
    # Parametros
    '\textbf{Parámetros}': ['Fecha y hora de Lanzamiento [UTC]', 'Objetivo de Altitud  [m]', 'Base de lanzamiento', 'Masa Globo [kg]', 'Masa Helio [kg]', 'Masa Payload [kg]', 'Presión Inicial [Pa]', 'Temperatura Inicial [K]'],
    # Valores para cada parametros
    '\textbf{Valor}': [date_launch+time_launch, target_burst_altitude, f'Lat: {lat_launch}, Lon: {lon_launch}', m_b, m_He, m_pl, P_launch, T_launch],
})

# Mostramos el datos de inicio
display(df)

# Exportar el dataframe a una tabla de LaTeX
with open('/home/osmin-ubuntu/proyecto_stratoballoon/document/table/02_datos_iniciales.tex', 'w') as f:
    f.write(df.to_latex(
            index=False,
            position="h",
            bold_rows=True,
            float_format="{:0.2f}".format,
            column_format="c"*len(df.columns),
            # longtable=True,
            escape=False,
            caption="Especificiaciones de globo-sonda e información de simulación de vuelo.",
            label="tab:datos_start_input",
            ))


,\textbf{Parámetros},\textbf{Valor}
0,Fecha y hora de Lanzamiento [UTC],2023-06-10 10:00:00
1,Objetivo de Altitud [m],30000
2,Base de lanzamiento,"Lat: 13.808825, Lon: -89.328988"
3,Masa Globo [kg],1.5
4,Masa Helio [kg],0.568913691920873
5,Masa Payload [kg],1.769
6,Presión Inicial [Pa],101325
7,Temperatura Inicial [K],288.15


/tmp/ipykernel_276/3808783882.py:14: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



## Tabla resumen simulación

In [40]:
# Función para aproximar
def aproximacion(columna):
    if columna.name == 'latitude' or columna.name == 'longitude':
        return columna.round(4)
    elif columna.name == 'altitud':
        columna = columna/1000
        return columna.round(2)
    elif columna.name == 'presion':
        columna = columna/100
        return columna.round(1)
    elif columna.name == 'temperatura':
        columna = columna-273.15
        return columna.round(1)  
    else:
        return columna.round(1)

columns = ['\textbf{t [s]}', '\textbf{Lat. ($\phi$)}', '\textbf{Long. ($\lambda$)}', '\textbf{Alt. (z) [km]}', '\textbf{$dz/dt$ [m/s]}', '\textbf{u [m/s]}',
           '\textbf{v [m/s]}', '\textbf{T [°C]}', '\textbf{P [hPa]}', '\textbf{$\rho$ [kg/m³]}', '\textbf{g [m/s²]}', '\textbf{d [m]}']


In [45]:
# Cargamos la BD a un DataFrame y leemos los datos del archivo .csv
df = pd.read_csv('../data/parameters_ascenso.csv', sep=',', header=0)
# Aproximamos
df = df.apply(aproximacion, axis=0)

#   Genero los limiadores de la Tabla a crear para no cargar todos los datos
limite_menor = round(len(df)/2)
limite_mayor = round(len(df)/2) + 6

large_dataset = pd.DataFrame(df.iloc[limite_menor - 3:limite_menor]).astype(str)
large_dataset[:] = '.'

large_dataset_2 = pd.DataFrame(df.iloc[limite_mayor:limite_mayor + 3]).astype(str)
large_dataset_2[:] = '.'

#   Concatena datos para solo tomar un fragmento inicial, intermedio y final
df_latex_table_simu = pd.concat([df.head(6),
                                 large_dataset,
                                 # Saco la mitad de dataframe e imprimo los siguientes 6 datos
                                 df.iloc[limite_menor:limite_mayor],
                                 large_dataset_2,
                                 df.tail(6)], ignore_index=True)

# Cambiamos nombres a las columnas
df_latex_table_simu.columns = columns

#   Mostramos Tabla creada
display(df_latex_table_simu)

#   Exportamos Tabla creada a LaTeX
with open('/home/osmin-ubuntu/proyecto_stratoballoon/document/table/02_tabla_simulacion_resumen_ascenso.tex', 'w') as f:
    f.write(
        df_latex_table_simu
        # .rename(columns={"mean": r"mean${2}$"})
        .to_latex(
            index=False,
            bold_rows=True,
            column_format="c"*len(df.columns),
            # longtable=True,
            escape=False,
            caption="Ascenso, fragmento de los datos generados en simulación.",
            label="tab:ascenso_frag_datos_generados",
        )
    )


,\textbf{t [s]},\textbf{Lat. ($\phi$)},\textbf{Long. ($\lambda$)},\textbf{Alt. (z) [km]},\textbf{$dz/dt$ [m/s]},\textbf{u [m/s]},\textbf{v [m/s]},\textbf{T [°C]},\textbf{P [hPa]},\textbf{$\rho$ [kg/m³]},\textbf{g [m/s²]},\textbf{d [m]}
0,0,13.8088,-89.329,0.5,0.0,0.0,0.4,11.7,954.2,1.2,9.8,1.9
1,1,13.8088,-89.329,0.51,1.9,0.0,0.4,11.7,954.0,1.2,9.8,1.9
2,2,13.8088,-89.329,0.51,2.8,0.0,0.4,11.7,953.8,1.2,9.8,1.9
3,3,13.8088,-89.329,0.51,3.2,0.0,0.4,11.7,953.4,1.2,9.8,1.9
4,4,13.8088,-89.329,0.51,3.3,0.0,0.4,11.7,953.0,1.2,9.8,1.9
5,5,13.8088,-89.329,0.52,3.3,0.0,0.4,11.6,952.7,1.2,9.8,1.9
6,.,.,.,.,.,.,.,.,.,.,.,.
7,.,.,.,.,.,.,.,.,.,.,.,.
8,.,.,.,.,.,.,.,.,.,.,.,.
9,3333,13.8752,-89.3277,12.72,4.2,-4.6,3.3,-56.5,173.3,0.3,9.8,3.0


/tmp/ipykernel_276/2109573600.py:33: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [44]:
# Cargamos la BD a un DataFrame y leemos los datos del archivo .csv
df = pd.read_csv('../data/parameters_descenso.csv', sep=',', header=0)
# Aproximamos
df = df.apply(aproximacion, axis=0)

#   Genero los limiadores de la Tabla a crear para no cargar todos los datos
limite_menor = round(len(df)/2)
limite_mayor = round(len(df)/2) + 6

large_dataset = pd.DataFrame(
    df.iloc[limite_menor - 3:limite_menor]).astype(str)
large_dataset[:] = '.'

large_dataset_2 = pd.DataFrame(
    df.iloc[limite_mayor:limite_mayor + 3]).astype(str)
large_dataset_2[:] = '.'

#   Concatena datos para solo tomar un fragmento inicial, intermedio y final
df_latex_table_simu = pd.concat([df.head(6),
                                 large_dataset,
                                 # Saco la mitad de dataframe e imprimo los siguientes 6 datos
                                 df.iloc[limite_menor:limite_mayor],
                                 large_dataset_2,
                                 df.tail(6)], ignore_index=True)

# Cambiamos nombres a las columnas
df_latex_table_simu.columns = columns

#   Mostramos Tabla creada
display(df_latex_table_simu)

#   Exportamos Tabla creada a LaTeX
with open('/home/osmin-ubuntu/proyecto_stratoballoon/document/table/02_tabla_simulacion_resumen_descenso.tex', 'w') as f:
    f.write(
        df_latex_table_simu
        # .rename(columns={"mean": r"mean${2}$"})
        .round(3)
        .to_latex(
            index=False,
            bold_rows=True,
            column_format="c"*len(df.columns),
            # longtable=True,
            escape=False,
            caption="Descenso, fragmento de los datos generados en simulación.",
            label="tab:descenso_frag_datos_generados"
        )
    )



,\textbf{t [s]},\textbf{Lat. ($\phi$)},\textbf{Long. ($\lambda$)},\textbf{Alt. (z) [km]},\textbf{$dz/dt$ [m/s]},\textbf{u [m/s]},\textbf{v [m/s]},\textbf{T [°C]},\textbf{P [hPa]},\textbf{$\rho$ [kg/m³]},\textbf{g [m/s²]},\textbf{d [m]}
0,6665,13.8699,-89.3244,29.99,6.5,-29.2,3.7,-46.6,12.0,0.0,9.7,0.0
1,6666,13.8698,-89.3244,30.0,-3.1,-29.2,3.7,-46.6,12.0,0.0,9.7,0.0
2,6667,13.8698,-89.3244,29.99,-12.5,-29.2,3.7,-46.6,12.0,0.0,9.7,0.0
3,6668,13.8698,-89.3244,29.97,-21.0,-29.2,3.6,-46.6,12.1,0.0,9.7,0.0
4,6669,13.8697,-89.3244,29.95,-28.1,-29.2,3.6,-46.6,12.1,0.0,9.7,0.0
5,6670,13.8697,-89.3244,29.92,-33.6,-29.2,3.5,-46.6,12.2,0.0,9.7,0.0
6,.,.,.,.,.,.,.,.,.,.,.,.
7,.,.,.,.,.,.,.,.,.,.,.,.
8,.,.,.,.,.,.,.,.,.,.,.,.
9,7843,13.8769,-89.3236,8.49,-9.2,-7.6,-11.3,-40.2,332.3,0.5,9.8,0.0


/tmp/ipykernel_276/1048474930.py:35: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



# Información de sesión

Se coloca la información de sesión al final de este notebook realizado, como una buena práctica. De esta forma, otras personas podrán ver qué versiones de librerías y dependencias se utilizaron para el análisis.

Además, esto permite ver si llegara alguien a tener un problema, puede compartir esta información con los desarrolladores de este notebook para que sea más fácil replicar el error y poder ayudar. 

In [43]:
# Importamos la librería
import session_info

# Lanzamos el comando
session_info.show()
